In [ ]:
import pickle

In [ ]:
tokenized_stores = {'en_train': [], 'en_dev': [], 'en_test': [], 'zh_train': [], 'zh_dev': [], 'zh_test': []}

In [ ]:

for key in tokenized_stores:
    file_name = "/content/sample_data/" + str(key)[3:] + "." + str(key)[0:2]
    load = open(file_name)
    sentences = load.read().split('\n')
    
    for sentence in sentences:
        token_store = sentence.split(' ')
        tokenized_stores[key].append(token_store)
        

In [ ]:
print(tokenized_stores['zh_train'][2])

['研究者', '确定', '了', '12', '项', '评估', '去', '脂', '体重', '变化', '情况', '的', '绝经期', '激素', '疗法', '随机', '试验', '.']


In [ ]:
train_size = len(tokenized_stores['en_train'])
dev_size = len(tokenized_stores['en_dev'])
test_size = len(tokenized_stores['en_test'])

In [ ]:


en_words = {}
zh_words = {}

for key in tokenized_stores:
    if str(key)[0] == 'e':
        # creating en_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in en_words:
                    en_words[word] += 1
                else:
                    en_words[word] = 1
    else:
        # creating hi_words
        for sentence in tokenized_stores[key]:
            for word in sentence:
                if word in zh_words:
                    zh_words[word] += 1
                else:
                    zh_words[word] = 1
                    
en_vocab = len(en_words)
zh_vocab = len(zh_words)
print("Number of Unique Words:")
print("> English:", str(en_vocab))
print("> Chinese:", str(zh_vocab))

Number of Unique Words:
> English: 38164
> Chinese: 34020


In [ ]:

t = {}

uniform = 1 / (en_vocab * zh_vocab)

In [ ]:
n_iters = 0
max_iters = 25

fine_tune = 1
has_converged = False

while n_iters < max_iters and has_converged == False:
    has_converged = True
    max_change = -1

    n_iters += 1
    count = {}
    total = {}
    for index in range(train_size):
        s_total = {}
        for en_word in tokenized_stores['en_train'][index]:
            s_total[en_word] = 0
            for zh_word in tokenized_stores['zh_train'][index]:
                if (en_word, zh_word) not in t:
                    t[(en_word, zh_word)] = uniform
                s_total[en_word] += t[(en_word, zh_word)]

        for en_word in tokenized_stores['en_train'][index]:
            for zh_word in tokenized_stores['zh_train'][index]:
                if (en_word, zh_word) not in count:
                    count[(en_word, zh_word)] = 0
                count[(en_word, zh_word)] += (t[(en_word, zh_word)] / s_total[en_word])

                if zh_word not in total:
                    total[zh_word] = 0
                total[zh_word] += (t[(en_word, zh_word)] / s_total[en_word])



    if fine_tune == 0:
      updated = {}
   
      for index in range(train_size):
          for zh_word in tokenized_stores['zh_train'][index]:
              for en_word in tokenized_stores['en_train'][index]:
                  if (en_word, zh_word) in updated:
                      continue
                  updated[(en_word, zh_word)] = 1
                  if abs(t[(en_word, zh_word)] - count[(en_word, zh_word)] / total[zh_word]) > 0.01:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, zh_word)] - count[(en_word, zh_word)] / total[zh_word]))
                  t[(en_word, zh_word)] = count[(en_word, zh_word)] / total[zh_word]

    elif fine_tune == 1:
 
      max_words = 1000
      n_zh_words = 0
      updates = 0

      for zh_word_tuples in sorted(zh_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
          zh_word = zh_word_tuples[0]
          n_zh_words += 1
          if n_zh_words > max_words:
              break
          n_en_words = 0
          for en_word_tuples in sorted(en_words.items(), key = lambda k:(k[1], k[0]), reverse = True):
              en_word = en_word_tuples[0]
              n_en_words += 1
              if n_en_words > max_words:
                  break
              if (en_word, zh_word) not in count or zh_word not in total:
                  continue
               
              else:
                  if abs(t[(en_word, zh_word)] - count[(en_word, zh_word)] / total[zh_word]) > 0.005:
                      has_converged = False
                      max_change = max(max_change, abs(t[(en_word, zh_word)] - count[(en_word, zh_word)] / total[zh_word]))
                  t[(en_word, zh_word)] = count[(en_word, zh_word)] / total[zh_word]

    print("Iteration " + str(n_iters) + " Completed, Maximum Change: " + str(max_change))


Iteration 1 Completed, Maximum Change: 0.20376036404370573
Iteration 2 Completed, Maximum Change: 0.3749189216287341
Iteration 3 Completed, Maximum Change: 0.2347414474468955
Iteration 4 Completed, Maximum Change: 0.14620234522828746
Iteration 5 Completed, Maximum Change: 0.08334033565664983
Iteration 6 Completed, Maximum Change: 0.06297144913688518
Iteration 7 Completed, Maximum Change: 0.0528870916541162
Iteration 8 Completed, Maximum Change: 0.04183652106048669
Iteration 9 Completed, Maximum Change: 0.0320971384198398
Iteration 10 Completed, Maximum Change: 0.02432988941236436
Iteration 11 Completed, Maximum Change: 0.019707657146795954
Iteration 12 Completed, Maximum Change: 0.017435692722802987
Iteration 13 Completed, Maximum Change: 0.015451156885838202
Iteration 14 Completed, Maximum Change: 0.01371928371508796
Iteration 15 Completed, Maximum Change: 0.012208097288418596
Iteration 16 Completed, Maximum Change: 0.010888563752155522
Iteration 17 Completed, Maximum Change: 0.009734

In [ ]:

limit = 40
for element in sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True):
  print(element)
  limit -= 1
  if limit <= 0:
    break

(('or', '或'), 0.7726819176949373)
(('safety', '安全性'), 0.7500700362309497)
(('cardiovascular', '心血管'), 0.7468134875684475)
(('2017', '2017'), 0.7112205685834544)
(('all', '所有'), 0.7046982270580987)
(('Health', 'Health'), 0.6983084389338453)
(('prespecified', '预设'), 0.6912945477433061)
(('secondary', '次要'), 0.6904431429301936)
(('chemotherapy', '化疗'), 0.6902246411458516)
(('2016', '2016'), 0.6893259410034063)
(('median', '中位'), 0.6870573323320014)
(('L1', 'L1'), 0.6842905773099405)
(('placebo', '安慰剂'), 0.6822322480622127)
(('acute', '急性'), 0.6755286315442989)
(('52', '52'), 0.6728076323698157)
(('T', 'T'), 0.6711851838077961)
(('PD', 'PD'), 0.6707055460847534)
(('National', 'National'), 0.6685973569291404)
(('and', '和'), 0.6682994163093748)
(('baseline', '基线'), 0.6677175533677872)
(('stroke', '卒'), 0.6669198539562238)
(('2015', '2015'), 0.6642072915941152)
((';', ';'), 0.6623895359464148)
(('mg', 'mg'), 0.6578949810127961)
(('ischemic', '缺血性'), 0.6570312560541093)
(('oral', '口服'), 0.6565

In [ ]:

file = open("translation_model.pkl","wb")
pickle.dump(t, file)
file.close()

In [ ]:

model_name = "translation_model.pkl"
pickle_in = open(model_name,"rb")
t = pickle.load(pickle_in)

In [ ]:
I = {}
for index in range(train_size):
    for en_id in range(len(tokenized_stores['en_train'][index])):
        length = len(tokenized_stores['en_train'][index])
        if length not in I:
            I[length] = {} # maps the positional difference to a tuple: (sum of t's, count)
        for zh_id in range(len(tokenized_stores['zh_train'][index])):
            if (zh_id - en_id) not in I[length]:
                I[length][(zh_id - en_id)] = [t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['zh_train'][index][zh_id])], 1]
            else:
                I[length][(zh_id - en_id)][0] += t[(tokenized_stores['en_train'][index][en_id], tokenized_stores['zh_train'][index][zh_id])]
                I[length][(zh_id - en_id)][1] += 1

In [ ]:

sentence_lengths = []
for key in I.keys():
    if key not in sentence_lengths:
        sentence_lengths.append(key)
sentence_lengths.sort()
print(sentence_lengths)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 79, 80, 83, 93, 96, 100, 107]


In [ ]:

 
p = {}
for key in I.keys():
    p[key] = {}
    sum_val = 0
    for diff in I[key].keys():
        p[key][diff] = I[key][diff][0] / I[key][diff][1]
        sum_val += p[key][diff]
    for diff in p[key].keys():
        p[key][diff] /= sum_val

In [ ]:
for index in range(train_size):
    length_en = len(tokenized_stores['en_train'][index])
    length_zh = len(tokenized_stores['zh_train'][index])
    if length_zh - length_en > 10 and length_en == 1:
        print("Length of English Sentence:", str(length_en))
        print("Length of Hindi Sentence:", str(length_zh))

Length of English Sentence: 1
Length of Hindi Sentence: 14


In [ ]:

init = {}
for length in p:
    max_prob = -1
    max_jump = 0
    for key in p[length].keys():
        if p[length][key] > max_prob:
            max_prob = p[length][key]
            max_jump = key
    init[length] = max_jump

In [ ]:
!pip install nltk

In [ ]:

bigrams = {}
unigrams = {}


def model(dataset_size, dataset_name):
    global bigrams
    global unigrams
    for index in range(dataset_size):
        token_A = ''
        for zh_token in tokenized_stores[dataset_name][index]:
            if zh_token not in unigrams:
                unigrams[zh_token] = 1
            else:
                unigrams[zh_token] += 1
            
            token_B = zh_token
            if (token_A, token_B) not in bigrams:
                bigrams[(token_A, token_B)] = 1
            else:
                bigrams[(token_A, token_B)] += 1
            token_A = token_B

model(train_size, 'zh_token')
model(dev_size, 'zh_dev')

bigram_count = len(bigrams)
unigram_count = len(unigrams)
print("Number of Unique Bigrams:", bigram_count)
print("Number of Unique Unigrams:", unigram_count)

Number of Unique Bigrams: 317170
Number of Unique Unigrams: 43851


In [ ]:
from itertools import permutations
import nltk

computed_sentences = []
total_BLEU = {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 7: 0}
null_BLEU_count = 0

sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)

def find_translation(en_token):
    for element in sorted_t:
        if element[0][0].lower() == en_token:
            return element[0][1]
    return ""

def get_prob(seq):
    if len(seq) < 2:
        return 1
    score = 0
    token_A = ''
    for hi_token in seq:
        token_B = hi_token
        if (token_A, token_B) not in bigrams:
            if token_B not in unigrams:
                continue
            else:
                score += unigrams[token_B] / unigram_count
        else:
            base_token_count = 0
            if token_A in unigrams:
                base_token_count = unigrams[token_A]
            score += (bigrams[(token_A, token_B)] + 1) / (base_token_count + unigram_count)
        token_A = token_B
    return score

count = 0
for index in range(test_size):
    if len(tokenized_stores['en_test'][index]) > 8 or len(tokenized_stores['en_test'][index]) < 2:
        continue

    translated_words = []
    for en_token in tokenized_stores['en_test'][index]:
        translation = find_translation(en_token)
        if translation != "":
            translated_words.append(translation)

    perm = permutations(translated_words)

    best_seq = translated_words
    best_prob = -1

    for seq in perm:
        prob = get_prob(seq)
        if prob > best_prob:
            best_prob = prob
            best_seq = seq

    BLEU_scores = []
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method1))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method2))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method3))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method5))
    BLEU_scores.append(nltk.translate.bleu_score.sentence_bleu([tokenized_stores['zh_test'][index]], best_seq, smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method7))

    for key in total_BLEU.keys():
        if key == 7:
            consider = 5
        else: consider = key - 1
        total_BLEU[key] += BLEU_scores[consider]
    
    if BLEU_scores[0] == 0:
        null_BLEU_count += 1
    
    count += 1
    print("Sentence Index: ", str(count))
    print("English Sentence:", str(tokenized_stores['en_test'][index]))
    print("Reference Chinese Sentence:", str(tokenized_stores['zh_test'][index]))
    print("Translated Sentence:", str(best_seq))
    print("Translation BLEU Scores", str(BLEU_scores))
    print()
    
    computed_sentences.append([tokenized_stores['en_test'][index], tokenized_stores['zh_test'][index], best_seq, BLEU_scores])

tested = count

Sentence Index:  1
English Sentence: ['Your', 'self-confidence', 'also', 'increases', 'with', 'teeth', '.']
Reference Hindi Sentence: ['你的'，'自信'，'也'，'增加'，'与'，'牙齿', '।']
Translated Sentence: ('।', '你的'，'自信'，'也'，'增加', '।', '也')
Translation BLEU Scores [0.04849815007763549, 0.19822631894109965, 0.07654112967106118, 0.18815926093992244, 0.09193101000946054, 0.2362891668915472]

Sentence Index:  2
English Sentence: ['Your', 'self-confidence', 'also', 'increases', 'with', 'teeth', '.']
Reference Hindi Sentence: [“细菌”、“停留”、“中间”、“我们的”、“牙龈”、“和”、“牙齿”, '।']
Translated Sentence: ('।', “细菌”、“停留”、“中间”、“我们的”, '।', '也')
Translation BLEU Scores [0.034052233956373766, 0.17820132316770917, 0.06770186228657868, 0.20529031524114758, 0.08738752502425957, 0.2638193908491072]

Sentence Index:  3
English Sentence: ['They', 'make', 'teeth', 'dirty', 'and', 'breath', 'stinky', '.']
Reference Hindi Sentence: [“他们”、“制造”、“牙齿”、“脏”、“和”、“呼吸”、“臭”, '।']
Translated Sentence: ('।', “脏”、“和”、“呼吸”、, '।', '也')
Translation BLE

In [ ]:
# Results:
import statistics
print("Number of Samples Tested Upon: " + str(tested))
print()

print("Average BLEU Score using Various Smoothing Functions (considering all test samples)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / tested))
print()
print("Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap)")
for key in total_BLEU:
    print("Method " + str(key) + ": " + str(total_BLEU[key] / (tested - null_BLEU_count)))

Number of Samples Tested Upon: 50

Average BLEU Score using Various Smoothing Functions (considering all test samples
Method 1: 0.04045334190563911
Method 2: 0.18790274652264405
Method 3: 0.08042839674590069
Method 4: 0.15545775044775517
Method 5: 0.019229913610012243
Method 7: 0.1878296463217631

Average BLEU Score using Various Smoothing Functions (considering test samples with at-least one word overlap
Method 1: 0.04127892031187664
Method 2: 0.1917374964516776
Method 3: 0.08206979259785784
Method 4: 0.15863035759975017
Method 5: 0.07735132483192965
Method 7: 0.19734686260161405
